In [ ]:
# good to go for AWS
import pandas as pd
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import re

import string
translator = str.maketrans('', '', string.punctuation)

def start_driver():
    options = webdriver.ChromeOptions()
    options.binary_location = '/opt/google/chrome/google-chrome'
    options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver', chrome_options=options)
    return driver

# sleep(100)

In [ ]:
# needs work
import pickle
from bs4 import BeautifulSoup
from time import sleep
import requests
import re

genre_tuples = [
    ('folk', 30, 'https://www.albumoftheyear.org/genre/5-folk/all/pitchfork/'), 
    ('indie_folk', 11, 'https://www.albumoftheyear.org/genre/16-indie-folk/all/'),
    ('folk_rock', 7, 'https://www.albumoftheyear.org/genre/14-folk-rock/all/pitchfork/'),
    ('alt_country', 8, 'https://www.albumoftheyear.org/genre/17-alt-country/all/'),
#     ('hip_hip', 48, 'https://www.albumoftheyear.org/genre/3-hip-hop/all/pitchfork/'),
#     ('pop', 15, 'https://www.albumoftheyear.org/genre/15-pop/all/pitchfork/')
]

def album_getter(tuples):
    
    for genre_tuple in tuples:
        name, pages, url  = genre_tuple
        
        albums = {}
        album_id = 1
        
        for page in range(1, pages):
            res = requests.get(url + str(page) + '/')
            print(name, page, res.status_code)
            soup = BeautifulSoup(res.content, 'lxml')
            for x in soup.find_all('div', {'class', 'albumListRow'}):
                
                score        = (x.find('div', {'class', 'scoreValue'}).text)
                album        = ((x.find('a').text.split('-')[1]).strip())
                artist       = ((x.find('a').text.split('-')[0]).strip())
                release_date = x.find('div', {'class', 'albumListDate'}).text

                albums[album_id] = {
                    'album_id'    : album_id,
                    'score'       : score,
                    'album'       : album,
                    'artist'      : artist,
                    'release_date': release_date
                }

                album_id = album_id + 1
            sleep(1)
            
        pickle.dump(albums, open(name + '_album_dict.pkl', 'wb'))

In [ ]:
import pandas as pd
import pickle
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import re
import string

translator = str.maketrans('', '', string.punctuation)

with open('all_folk_albums.pkl', 'rb') as f:
    all_folk_albums = pickle.load(f)

def lyric_getter(album_dict):
    
    song_id = 0
    
    base_url = 'https://genius.com/albums/'
    
    song_dictionary = {}
    
    bad_urls = []
    
    for album_id in album_dict:
        album         = album_dict[album_id]['album']
        artist        = album_dict[album_id]['artist']
        is_folk       = album_dict[album_id]['is_folk']
        is_folk_rock  = album_dict[album_id]['is_folk_rock']
        is_indie_folk = album_dict[album_id]['is_indie_folk']
        
        url_artist = artist.translate(translator).replace(' ', '-')
        url_album  = album.translate(translator).replace(' ', '-')
        
        album_href = base_url + url_artist + '/' + url_album + '/'
        
        album_result = requests.get(album_href)
        
        sleep(1)
        
        if album_result.status_code == 200:
            
            album_soup = BeautifulSoup(album_result.content, 'lxml')
            
            track_counts  = album_soup.find_all('div', {'class': 'chart_row-number_container chart_row-number_container--align_left'})
            track_listing = album_soup.find_all('div', {'class': 'chart_row-content'})
            
            for count, song in enumerate(track_listing):

                track_count = track_counts[count].find('span').text.replace('\n', '')
                song_title  = track_listing[count].find('h3').text.strip().split('\n')[0]
                href        = track_listing[count].find('a')['href']

                song_results = requests.get(href)
                song_soup    = BeautifulSoup(song_results.content, 'lxml')
                lyrics       = re.sub('(\\n){2}', '\\n', song_soup.find('p').text).replace('\n', ' n ')

                song_info = [
                    str(song_id),
                    str(album_id),
                    artist,
                    album,
                    album_href,
                    song_title,
                    track_count,
                    lyrics,
                    href,
                    str(is_folk),
                    str(is_folk_rock),
                    str(is_indie_folk),
                ]
                
                with open('song_lyrics.psv', 'a+') as f:
                    f.write('|'.join(song_info) + '\n')
                    
                song_dictionary[song_id] = {
                    'song_id'      : song_id,
                    'album_id'     : album_id,
                    'artist'       : artist,
                    'album'        : album,
                    'album_href'   : album_href,
                    'song_title'   : song_title,
                    'track_count'  : track_count,
                    'lyrics'       : lyrics,
                    'href'         : href,
                    'is_folk'      : is_folk,
                    'is_folk_rock' : is_folk_rock,
                    'is_indie_folk': is_indie_folk
                }
                    
                song_id = song_id + 1
                
                sleep(1)
                
        else:
            bad_urls.append(album_id)
            
    pickle.dump(song_dictionary, open('song_dictionary.pkl', 'wb'))
            
    pickle.dump(bad_urls, open('bad_urls.pkl', 'wb'))
    
sleep(100)

lyric_getter(all_folk_albums)

In [ ]:
def is_folk(cell):
    return 1

# folk_albums
folk_albums = pickle.load(open('folk_album_dict.pkl', 'rb'))

folk_df = pd.DataFrame(folk_albums).T

folk_df['is_folk'] = folk_df['album'].apply(is_folk)

# folk_rock_albums
rock_folk_albums  = pickle.load(open('folk_rock_album_dict.pkl', 'rb'))

folk_rock_df = pd.DataFrame(rock_folk_albums).T

folk_rock_df['is_folk_rock'] = folk_rock_df['album'].apply(is_folk)

# indie_folk_albums
indie_folk_albums = pickle.load(open('indie_folk_album_dict.pkl', 'rb'))

indie_folk_df = pd.DataFrame(indie_folk_albums).T

indie_folk_df['is_indie_folk'] = indie_folk_df['album'].apply(is_folk)

# Merge different folk album lists together
folk_df = pd.merge(left=folk_df,
                   right=folk_rock_df[['album', 'artist', 'is_folk_rock']],
                   on=['album', 'artist'],
                   how='outer')

folk_df = pd.merge(left=folk_df,
                   right=indie_folk_df[['album', 'artist', 'is_indie_folk']],
                   on=['album', 'artist'],
                   how='outer')

# fill nan values with 0
folk_df[['is_folk', 'is_folk_rock', 'is_indie_folk']] = folk_df[['is_folk', 'is_folk_rock', 'is_indie_folk']].fillna(0)

folk_df['album_id'] = folk_df.index

folk_df.fillna(0, inplace=True)

# convert to dict
all_folk_dict = folk_df.T.to_dict()

# save dict as pickle file
pickle.dump(all_folk_dict, open('all_folk_albums.pkl', 'wb'))